## Configurando o MySQL com Python

Neste momento, vamos configurar a integração entre o MySQL e Python. Isso nos permitirá executar consultas, inserções e outras operações no banco de dados MySQL usando o Python como nossa linguagem de programação principal.

In [1]:
import os
from dotenv import load_dotenv
import mysql.connector

# Carrega as variáveis do arquivo .env no ambiente de trabalho
load_dotenv()

# A função os.getenv é usada para obter o valor das variáveis de ambiente
host = os.getenv("DB_HOST")
user = os.getenv("DB_USERNAME")
password = os.getenv("DB_PASSWORD")

cnx = mysql.connector.connect(
    host=host,
    user=user,
    password=password
)

print(cnx)

ModuleNotFoundError: No module named 'dotenv'

Criaremos um cursor, que é um objeto utilizado para executar instruções SQL no contexto do Python. Essse cursor nos permitirá enviar consultas e comandos SQL para o banco de dados MySQL por meio da conexão estabelecida e obter os resultados de volta para serem processados em nosso código Python

In [73]:
cursor = cnx.cursor()

## Criando uma base de dados

O método `execute()` é usado para compilar uma instrução SQL

In [74]:
cursor.execute("create database if not exists dbprodutos;")

In [45]:
cursor.execute("show databases;")
for db in cursor:
    print(db)

('clubeDoLivro',)
('dbprodutos',)
('information_schema',)


In [46]:
cursor.execute("use dbprodutos")

## Criando tabela tb_livros

Antes de criar nossa tabela, vamos revisar as colunas necessárias que precisaremos incluir nela. Essa etapa é importante para garantir que nossa tabela seja projetada corretamente, atendendo aos requisitos de armazenamento e organização dos dados.

In [47]:
import pandas as pd

def_livros = pd.read_csv("/home/sheyla/Documents/Repositórios/pipeline-integrated/data/tabela_livros.csv")
def_livros.head()

,_id,Produto,Categoria do Produto,Preço,Frete,Data da Compra,Vendedor,Local da compra,Avaliação da compra,Tipo de pagamento,Quantidade de parcelas,Latitude,Longitude
0,65c9178189aa9220d3fdb847,Modelagem preditiva,livros,92.45,5.609697,2020-01-01,Thiago Silva,BA,1,cartao_credito,3,-13.29,-41.71
1,65c9178189aa9220d3fdb848,Iniciando em programação,livros,43.84,0.000000,2020-01-01,Mariana Ferreira,SP,5,cartao_credito,1,-22.19,-48.79
2,65c9178189aa9220d3fdb85a,Iniciando em programação,livros,63.25,3.894137,2022-01-01,Juliana Costa,RJ,5,cartao_credito,4,-22.25,-42.66
3,65c9178189aa9220d3fdb868,Ciência de dados com python,livros,86.13,5.273176,2021-02-01,Camila Ribeiro,RJ,4,cartao_credito,3,-22.25,-42.66
4,65c9178189aa9220d3fdb86a,Ciência de dados com python,livros,72.75,1.458158,2021-02-01,Beatriz Moraes,PR,4,cartao_credito,2,-24.89,-51.55


In [48]:
def_livros.columns

Index(['_id', 'Produto', 'Categoria do Produto', 'Preço', 'Frete',
       'Data da Compra', 'Vendedor', 'Local da compra', 'Avaliação da compra',
       'Tipo de pagamento', 'Quantidade de parcelas', 'Latitude', 'Longitude'],
      dtype='object')

In [49]:
def_livros.shape

(742, 13)

In [50]:
cursor.execute("""
    CREATE TABLE IF NOT EXISTS dbprodutos.tb_livros(
               id VARCHAR(100),
               Produto VARCHAR(100),
               Categoria_Produto VARCHAR(100),
               Preco FLOAT(10,2),
               Frete FLOAT(10,2),
               Data_Compra DATE,
               Vendedor VARCHAR(100),
               Local_Compra VARCHAR(100),
               Avaliacao_Compra INT,
               Tipo_Pagamento VARCHAR(100),
               Qntd_Parcelas INT,
               Latitude FLOAT(10,2),
               Longitude FLOAT(10,2),
               primary key(id)
    );
               
""")

**Selecionando a base de dados para verificar a tabela criada**

In [51]:
cursor.execute("use dbprodutos;")
cursor.execute("show tables;")

## Inserindo os dados do csv na tabela tb_livros

Para inserir os dados na tabela do MySQL, é necessário percorrer cada linha do DataFrame e transformá-las em tuplas. Essa abordagem permite que mapeemos os dados do DataFrame para as colunas correspondentes da tabela do MySQL de forma eficiente e precisa.

In [52]:
for i, row in def_livros.iterrows():
    print(tuple(row))

('65c9178189aa9220d3fdb847', 'Modelagem preditiva', 'livros', 92.45, 5.6096965236, '2020-01-01', 'Thiago Silva', 'BA', 1, 'cartao_credito', 3, -13.29, -41.71)
('65c9178189aa9220d3fdb848', 'Iniciando em programação', 'livros', 43.84, 0.0, '2020-01-01', 'Mariana Ferreira', 'SP', 5, 'cartao_credito', 1, -22.19, -48.79)
('65c9178189aa9220d3fdb85a', 'Iniciando em programação', 'livros', 63.25, 3.894137208, '2022-01-01', 'Juliana Costa', 'RJ', 5, 'cartao_credito', 4, -22.25, -42.66)
('65c9178189aa9220d3fdb868', 'Ciência de dados com python', 'livros', 86.13, 5.2731755368, '2021-02-01', 'Camila Ribeiro', 'RJ', 4, 'cartao_credito', 3, -22.25, -42.66)
('65c9178189aa9220d3fdb86a', 'Ciência de dados com python', 'livros', 72.75, 1.458157705, '2021-02-01', 'Beatriz Moraes', 'PR', 4, 'cartao_credito', 2, -24.89, -51.55)
('65c9178189aa9220d3fdb86d', 'Iniciando em programação', 'livros', 61.73, 0.0, '2021-02-01', 'Beatriz Moraes', 'SP', 4, 'cartao_credito', 1, -22.19, -48.79)
('65c9178189aa9220d3fdb8

In [53]:
lista_dados = [tuple(row) for i, row in def_livros.iterrows()]
lista_dados

[('65c9178189aa9220d3fdb847',
  'Modelagem preditiva',
  'livros',
  92.45,
  5.6096965236,
  '2020-01-01',
  'Thiago Silva',
  'BA',
  1,
  'cartao_credito',
  3,
  -13.29,
  -41.71),
 ('65c9178189aa9220d3fdb848',
  'Iniciando em programação',
  'livros',
  43.84,
  0.0,
  '2020-01-01',
  'Mariana Ferreira',
  'SP',
  5,
  'cartao_credito',
  1,
  -22.19,
  -48.79),
 ('65c9178189aa9220d3fdb85a',
  'Iniciando em programação',
  'livros',
  63.25,
  3.894137208,
  '2022-01-01',
  'Juliana Costa',
  'RJ',
  5,
  'cartao_credito',
  4,
  -22.25,
  -42.66),
 ('65c9178189aa9220d3fdb868',
  'Ciência de dados com python',
  'livros',
  86.13,
  5.2731755368,
  '2021-02-01',
  'Camila Ribeiro',
  'RJ',
  4,
  'cartao_credito',
  3,
  -22.25,
  -42.66),
 ('65c9178189aa9220d3fdb86a',
  'Ciência de dados com python',
  'livros',
  72.75,
  1.458157705,
  '2021-02-01',
  'Beatriz Moraes',
  'PR',
  4,
  'cartao_credito',
  2,
  -24.89,
  -51.55),
 ('65c9178189aa9220d3fdb86d',
  'Iniciando em progr

In [59]:
sql = "insert into dbprodutos.tb_livros values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);"
cursor.executemany(sql, lista_dados)
cnx.commit()

In [60]:
print(cursor.rowcount, "dados inseridos na tabela tb_livros")

742 dados inseridos


## Criando tabela tb_produtos_2021_em_diante

In [62]:
def_livros2 = pd.read_csv("/home/sheyla/Documents/Repositórios/pipeline-integrated/data/tabela2021_em_diante.csv")
def_livros2.head()

,_id,Produto,Categoria do Produto,Preço,Frete,Data da Compra,Vendedor,Local da compra,Avaliação da compra,Tipo de pagamento,Quantidade de parcelas,Latitude,Longitude
0,65c9178189aa9220d3fdb850,Xadrez de madeira,brinquedos,25.23,0.000000,2021-01-01,Thiago Silva,BA,5,cartao_credito,2,-13.29,-41.71
1,65c9178189aa9220d3fdb851,Impressora,eletronicos,322.04,14.732100,2021-01-01,João Souza,SP,3,cartao_credito,1,-22.19,-48.79
2,65c9178189aa9220d3fdb852,Mesa de centro,moveis,282.22,12.611805,2021-01-01,João Souza,SP,3,boleto,1,-22.19,-48.79
3,65c9178189aa9220d3fdb853,Tablet ABXY,eletronicos,1100.42,68.451348,2021-01-01,Beatriz Moraes,SP,1,cupom,1,-22.19,-48.79
4,65c9178189aa9220d3fdb854,Fogão,eletrodomesticos,791.81,42.444626,2021-01-01,Juliana Costa,SP,4,boleto,1,-22.19,-48.79


In [63]:
def_livros2.shape

(6574, 13)

In [64]:
cursor.execute("""
    CREATE TABLE IF NOT EXISTS dbprodutos.tb_produtos_2021_em_diante(
               id VARCHAR(100),
               Produto VARCHAR(100),
               Categoria_Produto VARCHAR(100),
               Preco FLOAT(10,2),
               Frete FLOAT(10,2),
               Data_Compra DATE,
               Vendedor VARCHAR(100),
               Local_Compra VARCHAR(100),
               Avaliacao_Compra INT,
               Tipo_Pagamento VARCHAR(100),
               Qntd_Parcelas INT,
               Latitude FLOAT(10,2),
               Longitude FLOAT(10,2),
               primary key(id)
    );
               
""")

In [65]:
cursor.execute("use dbprodutos;")
cursor.execute("show tables;")

## Inserindo os dados do csv na tabela tb_produtos_2021_em_diante

In [66]:
for i, row in def_livros2.iterrows():
    print(tuple(row))

('65c9178189aa9220d3fdb850', 'Xadrez de madeira', 'brinquedos', 25.23, 0.0, '2021-01-01', 'Thiago Silva', 'BA', 5, 'cartao_credito', 2, -13.29, -41.71)
('65c9178189aa9220d3fdb851', 'Impressora', 'eletronicos', 322.04, 14.7321002401, '2021-01-01', 'João Souza', 'SP', 3, 'cartao_credito', 1, -22.19, -48.79)
('65c9178189aa9220d3fdb852', 'Mesa de centro', 'moveis', 282.22, 12.6118050347, '2021-01-01', 'João Souza', 'SP', 3, 'boleto', 1, -22.19, -48.79)
('65c9178189aa9220d3fdb853', 'Tablet ABXY', 'eletronicos', 1100.42, 68.4513476183, '2021-01-01', 'Beatriz Moraes', 'SP', 1, 'cupom', 1, -22.19, -48.79)
('65c9178189aa9220d3fdb854', 'Fogão', 'eletrodomesticos', 791.81, 42.4446263246, '2021-01-01', 'Juliana Costa', 'SP', 4, 'boleto', 1, -22.19, -48.79)
('65c9178189aa9220d3fdb855', 'Violão', 'instrumentos musicais', 316.84, 17.1595536296, '2022-01-01', 'Pedro Gomes', 'SP', 5, 'cartao_credito', 2, -22.19, -48.79)
('65c9178189aa9220d3fdb856', 'Geladeira', 'eletrodomesticos', 2322.94, 121.27400572

In [67]:
lista_dados2 = [tuple(row) for i, row in def_livros2.iterrows()]
lista_dados2

[('65c9178189aa9220d3fdb850',
  'Xadrez de madeira',
  'brinquedos',
  25.23,
  0.0,
  '2021-01-01',
  'Thiago Silva',
  'BA',
  5,
  'cartao_credito',
  2,
  -13.29,
  -41.71),
 ('65c9178189aa9220d3fdb851',
  'Impressora',
  'eletronicos',
  322.04,
  14.7321002401,
  '2021-01-01',
  'João Souza',
  'SP',
  3,
  'cartao_credito',
  1,
  -22.19,
  -48.79),
 ('65c9178189aa9220d3fdb852',
  'Mesa de centro',
  'moveis',
  282.22,
  12.6118050347,
  '2021-01-01',
  'João Souza',
  'SP',
  3,
  'boleto',
  1,
  -22.19,
  -48.79),
 ('65c9178189aa9220d3fdb853',
  'Tablet ABXY',
  'eletronicos',
  1100.42,
  68.4513476183,
  '2021-01-01',
  'Beatriz Moraes',
  'SP',
  1,
  'cupom',
  1,
  -22.19,
  -48.79),
 ('65c9178189aa9220d3fdb854',
  'Fogão',
  'eletrodomesticos',
  791.81,
  42.4446263246,
  '2021-01-01',
  'Juliana Costa',
  'SP',
  4,
  'boleto',
  1,
  -22.19,
  -48.79),
 ('65c9178189aa9220d3fdb855',
  'Violão',
  'instrumentos musicais',
  316.84,
  17.1595536296,
  '2022-01-01',
  '

In [75]:
sql = "insert into dbprodutos.tb_produtos_2021_em_diante values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);"
cursor.executemany(sql, lista_dados2)
cnx.commit()

In [76]:
print(cursor.rowcount, "dados inseridos na tabela tb_produtos_2021_em_diante")

6574 dados inseridos na tabela tb_produtos_2021_em_diante


## Visualizando os dados inseridos

In [77]:
cursor.execute("select * from dbprodutos.tb_livros;")

for row in cursor:
    print(row)

('65c9178189aa9220d3fdb847', 'Modelagem preditiva', 'livros', 92.45, 5.61, datetime.date(2020, 1, 1), 'Thiago Silva', 'BA', 1, 'cartao_credito', 3, -13.29, -41.71)
('65c9178189aa9220d3fdb848', 'Iniciando em programação', 'livros', 43.84, 0.0, datetime.date(2020, 1, 1), 'Mariana Ferreira', 'SP', 5, 'cartao_credito', 1, -22.19, -48.79)
('65c9178189aa9220d3fdb85a', 'Iniciando em programação', 'livros', 63.25, 3.89, datetime.date(2022, 1, 1), 'Juliana Costa', 'RJ', 5, 'cartao_credito', 4, -22.25, -42.66)
('65c9178189aa9220d3fdb868', 'Ciência de dados com python', 'livros', 86.13, 5.27, datetime.date(2021, 2, 1), 'Camila Ribeiro', 'RJ', 4, 'cartao_credito', 3, -22.25, -42.66)
('65c9178189aa9220d3fdb86a', 'Ciência de dados com python', 'livros', 72.75, 1.46, datetime.date(2021, 2, 1), 'Beatriz Moraes', 'PR', 4, 'cartao_credito', 2, -24.89, -51.55)
('65c9178189aa9220d3fdb86d', 'Iniciando em programação', 'livros', 61.73, 0.0, datetime.date(2021, 2, 1), 'Beatriz Moraes', 'SP', 4, 'cartao_credi

In [78]:
cursor.close()

True

In [79]:
cnx.close()

In [2]:
import sys
sys.path.append("/home/sheyla/Documents/Repositórios/pipeline-integrated/venv/lib/python3/site-packages")
